In [1]:
import sys
import numpy as np
from keras.datasets import mnist

In [2]:
(x_train, y_train),(x_test,y_test) = mnist.load_data()

In [3]:
images, labels = (x_train[0:1000].reshape(1000,28*28), y_train[0:1000])

In [4]:
one_hot_labels = np.zeros((len(labels),10))

In [5]:
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

In [6]:
test_images = x_test.reshape(len(x_test),28*28) /255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

In [7]:
np.random.seed(1)

In [8]:
relu = lambda x:(x>0) * x
relu2deriv = lambda x: x>=0

In [9]:
alpha = 0.005
iterations = 350
hidden_size = 40
pixels_per_image = 784
num_labels = 10

In [10]:
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

In [11]:
for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1,weights_1_2)

        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) == \
                                        np.argmax(labels[i:i+1]))

        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)\
                                    * relu2deriv(layer_1)
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    sys.stdout.write("\r I:"+str(j)+ \
                     " Train-Err:" + str(error/float(len(images)))[0:5] +\
                     " Train-Acc:" + str(correct_cnt/float(len(images))))

<ipython-input-11-1396ca63bc1b>:9: RuntimeWarning: overflow encountered in square
  error += np.sum((labels[i:i+1] - layer_2) ** 2)


 I:31 Train-Err:nan Train-Acc:0.097

KeyboardInterrupt: 